In [1]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from qat.qpus import get_default_qpu
from qat.core import Batch, Job

from qat.fermion.transforms import transform_to_jw_basis
from qat.fermion.hamiltonians import make_embedded_model
from qat.fermion.circuits import make_shallow_circ, make_ldca_circ

from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation
from qat.fermion.chemistry import MolecularHamiltonian, MoleculeInfo
from qat.fermion.trotterisation import make_trotterisation_routine
from qat.fermion.chemistry.ucc import construct_ucc_ansatz, guess_init_params, get_hf_ket, get_cluster_ops
from qat.qpus import get_default_qpu

from qat.plugins import ScipyMinimizePlugin, MultipleLaunchesAnalyzer

In [2]:
import numpy as np

from qat.core import Term
from qat.fermion.hamiltonians import FermionHamiltonian

# We define an arbitrary fermionic Hamiltonian H_f
nqbits = 4
H_fermion_double = FermionHamiltonian(nqbits, [Term(1, "CCcc", [0, 1, 2, 3]), Term(-1, "CCcc", [3, 2, 1, 0])])

print(f"H_fermion = {H_fermion_double}")

H_fermion = 1 * (CCcc|[0, 1, 2, 3]) +
-1 * (CCcc|[2, 3, 0, 1])


In [3]:
# Using the Jordan-Wigner transform
H_sp = H_fermion_double.to_spin("jordan-wigner")


print(f"H_spin = {H_sp}")

H_spin = 0.125j * (YYYX|[0, 1, 2, 3]) +
-0.125j * (XXYX|[0, 1, 2, 3]) +
-0.125j * (YXYY|[0, 1, 2, 3]) +
(-0-0.125j) * (XYYY|[0, 1, 2, 3]) +
0.125j * (YXXX|[0, 1, 2, 3]) +
0.125j * (XYXX|[0, 1, 2, 3]) +
0.125j * (YYXY|[0, 1, 2, 3]) +
-0.125j * (XXXY|[0, 1, 2, 3])


In [4]:
nqbits = 6
H_fermion_triple = FermionHamiltonian(nqbits, [Term(1, "CCCccc", [0, 1, 2, 3, 4, 5]), Term(-1, "CCCccc", [5, 4, 3, 2, 1, 0])])

print(f"H_fermion = {H_fermion_triple}")

H_fermion = 1 * (CCCccc|[0, 1, 2, 3, 4, 5]) +
-1 * (CCCccc|[3, 4, 5, 0, 1, 2])


In [5]:
# Using the Jordan-Wigner transform
H_sp = H_fermion_triple.to_spin("jordan-wigner")


print(f"H_spin = {H_sp}")

H_spin = 0.03125j * (YXYXYX|[0, 1, 2, 3, 4, 5]) +
0.03125j * (YYXXYX|[0, 1, 2, 3, 4, 5]) +
-0.03125j * (XXXXYX|[0, 1, 2, 3, 4, 5]) +
0.03125j * (XYYXYX|[0, 1, 2, 3, 4, 5]) +
-0.03125j * (YXXXYY|[0, 1, 2, 3, 4, 5]) +
0.03125j * (YYYXYY|[0, 1, 2, 3, 4, 5]) +
-0.03125j * (XXYXYY|[0, 1, 2, 3, 4, 5]) +
(-0-0.03125j) * (XYXXYY|[0, 1, 2, 3, 4, 5]) +
0.03125j * (YXXXXX|[0, 1, 2, 3, 4, 5]) +
-0.03125j * (YYYXXX|[0, 1, 2, 3, 4, 5]) +
0.03125j * (XXYXXX|[0, 1, 2, 3, 4, 5]) +
0.03125j * (XYXXXX|[0, 1, 2, 3, 4, 5]) +
0.03125j * (YXYXXY|[0, 1, 2, 3, 4, 5]) +
0.03125j * (YYXXXY|[0, 1, 2, 3, 4, 5]) +
(-0-0.03125j) * (XXXXXY|[0, 1, 2, 3, 4, 5]) +
0.03125j * (XYYXXY|[0, 1, 2, 3, 4, 5]) +
-0.03125j * (YXXYYX|[0, 1, 2, 3, 4, 5]) +
0.03125j * (YYYYYX|[0, 1, 2, 3, 4, 5]) +
-0.03125j * (XXYYYX|[0, 1, 2, 3, 4, 5]) +
-0.03125j * (XYXYYX|[0, 1, 2, 3, 4, 5]) +
-0.03125j * (YXYYYY|[0, 1, 2, 3, 4, 5]) +
(-0-0.03125j) * (YYXYYY|[0, 1, 2, 3, 4, 5]) +
0.03125j * (XXXYYY|[0, 1, 2, 3, 4, 5]) +
-0.03125j * (XYYYYY|[0, 1

In [6]:
nqbits = 8
H_fermion_quadriple = FermionHamiltonian(nqbits, [Term(1, "CCCCcccc", [0, 1, 2, 3, 4, 5, 6, 7]), Term(-1, "CCCCcccc", [7, 6, 5, 4, 3, 2, 1, 0])])

print(f"H_fermion = {H_fermion_quadriple}")

H_fermion = 1 * (CCCCcccc|[0, 1, 2, 3, 4, 5, 6, 7]) +
-1 * (CCCCcccc|[4, 5, 6, 7, 0, 1, 2, 3])


In [7]:
# Using the Jordan-Wigner transform
H_sp = H_fermion_quadriple.to_spin("jordan-wigner")


print(f"H_spin = {H_sp}")

H_spin = -0.0078125j * (YXYYYXYX|[0, 1, 2, 3, 4, 5, 6, 7]) +
0.0078125j * (YXXXYXYX|[0, 1, 2, 3, 4, 5, 6, 7]) +
-0.0078125j * (YYYXYXYX|[0, 1, 2, 3, 4, 5, 6, 7]) +
-0.0078125j * (YYXYYXYX|[0, 1, 2, 3, 4, 5, 6, 7]) +
0.0078125j * (XXYXYXYX|[0, 1, 2, 3, 4, 5, 6, 7]) +
0.0078125j * (XXXYYXYX|[0, 1, 2, 3, 4, 5, 6, 7]) +
-0.0078125j * (XYYYYXYX|[0, 1, 2, 3, 4, 5, 6, 7]) +
0.0078125j * (XYXXYXYX|[0, 1, 2, 3, 4, 5, 6, 7]) +
0.0078125j * (YXYXYXYY|[0, 1, 2, 3, 4, 5, 6, 7]) +
0.0078125j * (YXXYYXYY|[0, 1, 2, 3, 4, 5, 6, 7]) +
(-0-0.0078125j) * (YYYYYXYY|[0, 1, 2, 3, 4, 5, 6, 7]) +
0.0078125j * (YYXXYXYY|[0, 1, 2, 3, 4, 5, 6, 7]) +
0.0078125j * (XXYYYXYY|[0, 1, 2, 3, 4, 5, 6, 7]) +
-0.0078125j * (XXXXYXYY|[0, 1, 2, 3, 4, 5, 6, 7]) +
0.0078125j * (XYYXYXYY|[0, 1, 2, 3, 4, 5, 6, 7]) +
0.0078125j * (XYXYYXYY|[0, 1, 2, 3, 4, 5, 6, 7]) +
-0.0078125j * (YXYXYXXX|[0, 1, 2, 3, 4, 5, 6, 7]) +
-0.0078125j * (YXXYYXXX|[0, 1, 2, 3, 4, 5, 6, 7]) +
0.0078125j * (YYYYYXXX|[0, 1, 2, 3, 4, 5, 6, 7]) +
-0.0078125

# You can play it here for the number of CNOT

In [8]:
import numpy as np
import scipy
from numpy import binary_repr
from qat.fermion import ElectronicStructureHamiltonian
from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation
from qat.fermion.chemistry.ucc import (
    convert_to_h_integrals,
    transform_integrals_to_new_basis,
)
from qat.fermion.chemistry.ucc_deprecated import (
    get_active_space_hamiltonian,
    get_cluster_ops_and_init_guess,
)
from qat.fermion.transforms import (
    get_bk_code,
    get_jw_code,
    get_parity_code,
    recode_integer,
    transform_to_bk_basis,
    transform_to_jw_basis,
    transform_to_parity_basis,
)

from openvqe.common_files.generator_excitations import (
    singlet_gsd,
    singlet_sd,
    singlet_upccgsd,
    spin_complement_gsd,
    spin_complement_gsd_twin,
    uccsd,
)

In [9]:
import scipy.optimize
from qat.fermion.chemistry.ucc_deprecated import build_ucc_ansatz
from qat.lang.AQASM import Program
from qat.qpus import get_default_qpu
from openvqe.common_files.circuit import efficient_fermionic_ansatz, efficient_qubit_ansatz, count



def ucc_action(theta_current, hamiltonian_sp, cluster_ops_sp, hf_init_sp):
    """
    It maps the exponential of cluster operators ("cluster_ops_sp") associated by their parameters ("theta_current")
    using the CNOTS-staircase method, which is done by "build_ucc_ansatz" which creates the circuit on the top of
    the HF-state ("hf_init_sp"). Then, this function also calculates the expected value of the hamiltonian ("hamiltonian_sp").

    Parameters
    ----------
    theta_current: List<float>
        the Parameters of the cluster operators
    
    hamiltonian_sp: Hamiltonian
            Hamiltonian in the spin representation
        
    cluster_ops_sp: list[Hamiltonian]
        list of spin cluster operators
    
    hf_init_sp: int
        the integer corresponds to the hf_init (The Hartree-Fock state in integer representation) obtained by using
        "qat.fermion.transforms.record_integer".
    
    Returns
    --------
        res.value: float
            the resulted energy

    """
    qpu = 0
    prog = 0
    reg = 0
    qpu = get_default_qpu()
    prog = Program()
    reg = prog.qalloc(hamiltonian_sp.nbqbits)
    qrout = 0
    for n_term, (term, theta_term) in enumerate(zip(cluster_ops_sp, theta_current)):
        init = hf_init_sp if n_term == 0 else 0
        qprog = build_ucc_ansatz([term], init, n_steps=1)
        prog.apply(qprog([theta_term]), reg)
    circ = prog.to_circ()
    
    return circ



In [10]:
import numpy as np
import matplotlib.pyplot as plt
from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation
from qat.fermion.transforms import transform_to_jw_basis, get_jw_code, recode_integer
from openvqe.common_files.circuit import efficient_fermionic_ansatz, efficient_qubit_ansatz, count 

transformation, code = transform_to_jw_basis, get_jw_code

model = H_sp
qbits = model.nbqbits
n_electrons = 4

ket_hf_init = get_hf_ket(n_electrons, nqbits=nqbits)


cluster_ops = get_cluster_ops(n_electrons, nqbits=nqbits)
print(cluster_ops)

# Express the cluster operator in spin terms
cluster_ops_sp = [transformation(t_o) for t_o in cluster_ops]

hf_init_sp = recode_integer(ket_hf_init, code(nqbits))



n_steps =1 
prog = Program()

theta_0 = [np.random.uniform(0, 2*np.pi) for i in range(len(cluster_ops) * n_steps)]
theta = [prog.new_var(float, f"\\theta_{i}") for i in range(len(theta_0))]


circuit_ucc = ucc_action(theta, model, cluster_ops_sp, hf_init_sp)

circuit_ucc.display()



[Observable(terms=[FermionicTerm(_coeff=TNumber(is_abstract=False, type=5, int_p=None, double_p=None, string_p=None, matrix_p=None, serialized_p=None, complex_p=ComplexNumber(re=0.0, im=1.0)), op='Cc', qbits=[7, 3], _do_validity_check=True), FermionicTerm(_coeff=TNumber(is_abstract=False, type=5, int_p=None, double_p=None, string_p=None, matrix_p=None, serialized_p=None, complex_p=ComplexNumber(re=-0.0, im=-1.0)), op='Cc', qbits=[3, 7], _do_validity_check=True)], nbqbits=8, constant_coeff=0.0, ising=None), Observable(terms=[FermionicTerm(_coeff=TNumber(is_abstract=False, type=5, int_p=None, double_p=None, string_p=None, matrix_p=None, serialized_p=None, complex_p=ComplexNumber(re=0.0, im=1.0)), op='Cc', qbits=[7, 1], _do_validity_check=True), FermionicTerm(_coeff=TNumber(is_abstract=False, type=5, int_p=None, double_p=None, string_p=None, matrix_p=None, serialized_p=None, complex_p=ComplexNumber(re=-0.0, im=-1.0)), op='Cc', qbits=[1, 7], _do_validity_check=True)], nbqbits=8, constant_c

In [11]:
from openvqe.common_files.circuit import count

gate = circuit_ucc.ops
cnot = count("CNOT", gate)
print(cnot)


1312
